# 음료 crawling

In [1]:
import os
import sys
from selenium import webdriver
import chromedriver_autoinstaller

from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from bs4 import BeautifulSoup
import re

from selenium.webdriver.common.action_chains import ActionChains

In [2]:
def get_drink():
    #크롬드라이버 실행
    chromedriver_dir = r'C:/Users/LG/Downloads/chromedriver_win32/chromedriver.exe'
    driver = webdriver.Chrome(chromedriver_dir) #, chrome_options=options)

    #드라이버에 url을 넣음
    driver.get('https://www.mcdonalds.co.kr/kor/menu/list.do')

    #메뉴바로 마우스 이동
    menu_bar=driver.find_element_by_xpath('/html/body/div[2]/header/div/nav/div/ul')
    ActionChains(driver).move_to_element(menu_bar).perform()

    #음료 리스트 접속
    driver.find_element_by_xpath('/html/body/div[2]/header/div/nav/div/ul/li[1]/ul/li[6]/a').click()
    driver.implicitly_wait(3)
    driver.find_element_by_xpath('//*[@id="container"]/div[1]/div[2]/div/ul/li[2]/a').click()

    #메뉴 더보기 버튼 클릭
    while True:
        try:
            driver.find_element_by_xpath('''//*[@id="btnMore"]/button''').click()
        except:
            break
    #로딩완료 대기
    driver.implicitly_wait(3)

    #음료명 받아오기
    soup = BeautifulSoup(driver.page_source,'html.parser')
    drink_list=[]
    list = soup.find(id='menuList').find_all('strong')
    for i in list:
        drink_list.append(i.get_text())

    #음료 이미지 받아오기
    drink_img=[]
    list2 = soup.find(id='menuList').find_all('img')
    for i in list2:
        drink_img.append('https://www.mcdonalds.co.kr'+i['src'])

    result=[]
    i=0
    while i<len(drink_list):
        value=[]
        value.append(drink_list[i])
        value.append(drink_img[i])
        result.append(value)
        i+=1
    return result

# 음료정보DB UPDATE

In [3]:
import pymysql

In [4]:
def update_data(input_list):
    #DB연결하여 커서 얻기
    conn = pymysql.connect(host='localhost', user='root', password='0000', db='barcode', charset='utf8')
    cur = conn.cursor() 
    
    #2차원 리스트에서 음료명, 이미지 리스트 분리
    drink_list = []
    image_list = []
    for i in input_list:
        drink_list.append(i[0])
        image_list.append(i[1])
        
    # DB drink 데이터 얻기
    sql = "select * from drink_list"
    cur.execute(sql)

    # fetchall()함수 통해 모든 행 저장
    rows = cur.fetchall();
    DBlist = []

    # 모든 행 중 젤 앞 menu 열만 저장
    for i in rows:
        DBlist.append(i[0])
    del_list = []

    # DB에만 있고 크롤링 리스트에 없는 것 추출
    for i in DBlist:
        if i not in drink_list:
            del_list.append(i)

    # 기존 데이터 크롤링한 데이터에 없는 DB 데이터 삭제
    sql = "delete from drink_list where menu = %s"
    for i in range(len(del_list)):
        cur.execute(sql, (del_list[i]))

    # 삽입문(ignore 추가로 중복 무시하고 삽입)
    sql = "insert ignore into drink_list(drink, img) values(%s, %s)"
    for i in range(len(drink_list)):
        cur.execute(sql, (drink_list[i], image_list[i]))

    # DB에 반영
    conn.commit()

    # DB연결 종료
    conn.close()

In [5]:
update_data(get_drink())

# ---------------------------끝--------------------------